# AdaIN Style Transfer

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications import VGG19
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, UpSampling2D, Input
# import tensorflow_addons as tfa

import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

In [2]:
print("TensorFlow version:", tf.__version__)
print("GPU available:", tf.config.list_physical_devices('GPU'))

TensorFlow version: 2.15.0
GPU available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


## Load and Preprocess Images

In [3]:
def load_and_process_image(image_path, target_size=(256, 256)):
    img = Image.open(image_path).convert('RGB')
    img = img.resize(target_size)
    img = np.array(img, dtype=np.float32)
    img = np.expand_dims(img, axis=0)  # Add batch dimension
    img /= 255.0  # Normalize
    return img

# Example usage
content_image = load_and_process_image("./data/moss-forest.jpg")
style_image = load_and_process_image("./data/hokusai-fuji.jpg")

## Build Pretrained VGG-19 Encoder

In [4]:
def build_vgg_encoder():
    vgg = VGG19(weights="imagenet", include_top=False)
    vgg.trainable = False
    selected_layers = ["block1_conv1", "block2_conv1", "block3_conv1", "block4_conv1"]
    outputs = [vgg.get_layer(name).output for name in selected_layers]
    model = Model(inputs=vgg.input, outputs=outputs)
    return model

encoder = build_vgg_encoder()

2025-02-11 15:13:35.872365: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2 Pro
2025-02-11 15:13:35.872426: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2025-02-11 15:13:35.872438: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2025-02-11 15:13:35.872502: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-02-11 15:13:35.872538: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


80134624/80134624 [==============================] - 76s 1us/step


## Implement Adaptive Instance Normalization (AdaIN)

In [18]:
def adain(content_features, style_features, epsilon=1e-5):
    content_mean, content_var = tf.nn.moments(content_features, axes=[1,2], keepdims=True)
    style_mean, style_var = tf.nn.moments(style_features, axes=[1,2], keepdims=True)

    content_std = tf.sqrt(content_var + epsilon)
    style_std = tf.sqrt(style_var + epsilon)

    normalized_content = (content_features - content_mean) / content_std
    stylized_content = normalized_content * style_std + style_mean
    # return stylized_content
    return (content_features - content_mean) / content_std * style_std + style_mean

## Build a Small Decoder Network

In [6]:
def build_decoder():
    inputs = Input(shape=(None, None, 512))
    
    x = UpSampling2D()(inputs)
    x = Conv2D(256, (3,3), activation="relu", padding="same")(x)
    x = UpSampling2D()(x)
    x = Conv2D(128, (3,3), activation="relu", padding="same")(x)
    x = UpSampling2D()(x)
    x = Conv2D(64, (3,3), activation="relu", padding="same")(x)
    x = Conv2D(3, (3,3), activation="sigmoid", padding="same")(x)  # Sigmoid for output between 0-1

    return Model(inputs, x)

decoder = build_decoder()

## Define the Training Losses

In [7]:
def gram_matrix(features):
    features = tf.reshape(features, [features.shape[0], -1, features.shape[-1]])
    return tf.matmul(features, features, transpose_a=True) / features.shape[1]

def compute_loss(content_features, style_features, generated_features):
    content_loss = tf.reduce_mean(tf.square(generated_features - content_features))
    
    generated_gram = gram_matrix(generated_features)
    style_gram = gram_matrix(style_features)
    style_loss = tf.reduce_mean(tf.square(generated_gram - style_gram))

    return content_loss + 1e3 * style_loss

## Train the Model

In [22]:
optimizer = keras.optimizers.legacy.Adam(learning_rate=0.001)

@tf.function
def train_step(content_image, style_image):
    with tf.GradientTape() as tape:
        tape.watch(decoder.trainable_variables)  # Ensure tracking

        content_features = encoder(content_image)[-1]
        style_features = encoder(style_image)[-1]

        stylized_features = adain(content_features, style_features)
        generated_image = decoder(stylized_features)

        generated_features = encoder(generated_image)[-1]
        loss = compute_loss(content_features, style_features, generated_features)

    grads = tape.gradient(loss, decoder.trainable_variables)

    # ✅ Convert tensor to actual values
    grad_norms = [tf.norm(g).numpy() if g is not None else 0.0 for g in grads]
    print(f"Epoch Gradient Norms: {grad_norms}")

    optimizer.apply_gradients(zip(grads, decoder.trainable_variables))
    return loss

In [23]:
decoder_weights_before = decoder.get_weights()

# Train for a few epochs
for epoch in range(100):
    loss = train_step(content_image, style_image)

decoder_weights_after = decoder.get_weights()

# Compare weights before and after training
for i in range(len(decoder_weights_before)):
    print(f"Layer {i} weight change: {np.sum(decoder_weights_after[i] - decoder_weights_before[i])}")

AttributeError: in user code:

    File "/var/folders/_r/k49wtlq94ydc3vksghsj67ym0000gn/T/ipykernel_13263/3635838931.py", line 20, in train_step  *
        grad_norms = [tf.norm(g).numpy() if g is not None else 0.0 for g in grads]

    AttributeError: 'SymbolicTensor' object has no attribute 'numpy'


## Stylize a New Image

In [ ]:
def stylize_image(content_path, style_path):
    content_image = load_and_process_image(content_path)
    style_image = load_and_process_image(style_path)

    content_features = encoder(content_image)[-1]
    style_features = encoder(style_image)[-1]

    stylized_features = adain(content_features, style_features)
    generated_image = decoder(stylized_features)

    return np.clip(generated_image.numpy()[0] * 255, 0, 255).astype("uint8")

output_image = stylize_image("content.jpg", "style.jpg")
Image.fromarray(output_image).show()